# PY SPARK - Missing Data

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('miss').getOrCreate()

In [3]:
df = spark.read.csv('vendedores.csv', inferSchema=True,
                    header=True)

In [4]:
df.show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|    Sam|  200|
|emp2|Charlie| null|
|Emp3|   null| null|
|emp4|   Tina|  600|
|emp5|   null|  124|
|emp6|Vanessa|  243|
+----+-------+-----+



In [5]:
# RETIRAR A LINHA TODA CASO ELA TENHA MISSING DATA:
df.na.drop().show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|    Sam|  200|
|emp4|   Tina|  600|
|emp6|Vanessa|  243|
+----+-------+-----+



In [6]:
# THRESH: (seleciona a patir de quantos NA pode ter na linha/
#          para q ela nao seja apagada)
df.na.drop(thresh=2).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|    Sam|  200|
|emp2|Charlie| null|
|emp4|   Tina|  600|
|emp5|   null|  124|
|emp6|Vanessa|  243|
+----+-------+-----+



In [7]:
# OUTROS JEITOS
df.na.drop(how='all').show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|    Sam|  200|
|emp2|Charlie| null|
|Emp3|   null| null|
|emp4|   Tina|  600|
|emp5|   null|  124|
|emp6|Vanessa|  243|
+----+-------+-----+



In [8]:
df.na.drop(subset=['Sales']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|    Sam|  200|
|emp4|   Tina|  600|
|emp5|   null|  124|
|emp6|Vanessa|  243|
+----+-------+-----+



In [9]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: integer (nullable = true)



In [10]:
df.na.fill(0).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|    Sam|  200|
|emp2|Charlie|    0|
|Emp3|   null|    0|
|emp4|   Tina|  600|
|emp5|   null|  124|
|emp6|Vanessa|  243|
+----+-------+-----+



In [11]:
df.na.fill('FILL VALUE', subset=['Name']).show()

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|       Sam|  200|
|emp2|   Charlie| null|
|Emp3|FILL VALUE| null|
|emp4|      Tina|  600|
|emp5|FILL VALUE|  124|
|emp6|   Vanessa|  243|
+----+----------+-----+



In [12]:
# SUBSTITUINDO VALORES N.A. PELA MEDIA 
from pyspark.sql.functions import mean

In [13]:
mean_val = df.select(mean(df['Sales'])).collect()

In [14]:
# to dict
mean_sales = mean_val[0][0]

In [15]:
mean_sales

291.75

In [16]:
df.na.fill(mean_sales,['Sales']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|    Sam|  200|
|emp2|Charlie|  291|
|Emp3|   null|  291|
|emp4|   Tina|  600|
|emp5|   null|  124|
|emp6|Vanessa|  243|
+----+-------+-----+



In [17]:
# OU, FAZENDO TUDO NA MESMA LINHA DE CODIGO:
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0],['Sales']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|    Sam|  200|
|emp2|Charlie|  291|
|Emp3|   null|  291|
|emp4|   Tina|  600|
|emp5|   null|  124|
|emp6|Vanessa|  243|
+----+-------+-----+

